# Implementing dual numbers is with matrices
The dual number a+bϵ can be encoded as:
$$
\left(\begin{array}{ll}{a} & {b} \\ {0} & {a}\end{array}\right)
$$
This encoding has the properties ϵ≠0 and ϵ2=0:
$$
\left(\begin{array}{ll}{0} & {1} \\ {0} & {0}\end{array}\right)^{2}=\left(\begin{array}{ll}{0} & {0} \\ {0} & {0}\end{array}\right)
$$
Furthermore, they add, multiply and divide like dual numbers:
$$
\left(\begin{array}{ll}{3} & {1} \\ {0} & {3}\end{array}\right)\left(\begin{array}{ll}{3} & {1} \\ {0} & {3}\end{array}\right)=\left(\begin{array}{ll}{9} & {6} \\ {0} & {9}\end{array}\right)
$$

# Extend to higher-order derivatives
To compute second derivatives, we need to carry out the Taylor series expansion one step further. So instead of $ϵ^2$=0, we need $ϵ^3$=0 and ϵ≠$ϵ^2$. Then we get numbers of the form a+bϵ+c$ϵ^2$.Now we get
$$
a+b \epsilon+c \epsilon^{2}=\left(\begin{array}{lll}{a} & {b} & {c} \\ {0} & {a} & {b} \\ {0} & {0} & {a}\end{array}\right)
$$
It encodes the properties we want:
$$
\left(\begin{array}{lll}{0} & {1} & {0} \\ {0} & {0} & {1} \\ {0} & {0} & {0}\end{array}\right)^{2}=\left(\begin{array}{lll}{0} & {0} & {1} \\ {0} & {0} & {0} \\ {0} & {0} & {0}\end{array}\right) \quad\left(\begin{array}{lll}{0} & {1} & {0} \\ {0} & {0} & {1} \\ {0} & {0} & {0}\end{array}\right)^{3}=\left(\begin{array}{lll}{0} & {0} & {0} \\ {0} & {0} & {0} \\ {0} & {0} & {0}\end{array}\right)
$$
If we want 3rd derivatives, we need a 4 x 4 matrix:
$$
a+b \epsilon+c \epsilon^{2}+d \epsilon^{3}=\left(\begin{array}{llll}{a} & {b} & {c} & {d} \\ {0} & {a} & {b} & {c} \\ {0} & {0} & {a} & {b} \\ {0} & {0} & {0} & {a}\end{array}\right)
$$
We can extend this technique to any order derivative you want.

# Pseudocode
## (if we want to get the 2nd derivative of $y = x^2$ at x = 3)

In [10]:
import numpy as np

# Dual class to get the dual number
class Dual():
    def __init__(self, order_of_derivative, x_value):
        self.order_of_derivative = order_of_derivative
        self.x_value = x_value
        
    def get_matrix(self):
        n = self.order_of_derivative
        A = self.x_value*np.identity(n)
        A[0, n-1] = 1
        return A

In [17]:
import numpy as np
from numpy.linalg import matrix_power
#from Dual import Dual
#from elemFunctions import elemFunctions

A = Dual(2, 3)
A_matrix = A.get_matrix()

Y = matrix_power(A_matrix, 2)
print(Y)
second_derivative = Y[0,1]
print(second_derivative)

[[9. 6.]
 [0. 9.]]
6.0


# Problems
We can use this matrix to do easy operations, but not for more complicated operations, like sin, exp, and so on, so we need to overload these elemental functions for Dual Number. Wht I am think is using Taylor Series to translate the sin, exp, ... operations to simple operations, and then do the Numpy Martix operations.
Take $e^{x}$ for example
$$
\begin{aligned} e^{x} &=1+x+\frac{x^{2}}{2 !}+\frac{x^{3}}{3 !}+\frac{x^{4}}{4 !}+\cdots \\ &=\sum_{n=0}^{\infty} \frac{x^{n}}{n !} \end{aligned}
$$
In the code, we can not add it to infinity, so we have to do the approximation. So for $e^{x}$, the elemental function shuold be something like this

In [23]:
import numpy as np
from numpy.linalg import matrix_power

def exp():
    e_to_2 = 0
    for i in range(20): # you can set it higher than 20 to better estimate the derivative
        e_to_2 += matrix_power(A_matrix, i)/math.factorial(i)
    return e_to_2
    
second_derivative = exp()[0, 1]
print(second_derivative)

20.08553691196314


And we can do the same thing for sin, cos, .... So the biggest work is to implement the elemental functions based on the Taylor series:

$$
\begin{aligned} \cos x &=1-\frac{x^{2}}{2 !}+\frac{x^{4}}{4 !}-\frac{x^{6}}{6 !}+\frac{x^{8}}{8 !}-\cdots \\ &=\sum_{n=0}^{\infty}(-1)^{n} \frac{x^{2 n}}{(2 n) !} \end{aligned}
$$

$$
\begin{aligned}\sin x &=x-\frac{x^{3}}{3 !}+\frac{x^{5}}{5 !}-\frac{x^{7}}{7 !}+\frac{x^{9}}{9 !}-\cdots \\ &=\sum_{n=1}^{\infty}(-1)^{(n-1)} \frac{x^{2 n-1}}{(2 n-1) !} \stackrel{\text { or }}{(2 n-1) !} \quad \stackrel{\text {or }}{=} \sum_{n=0}^{\infty}(-1)^{n} \frac{x^{2 n+1}}{(2 n+1) !} \end{aligned}
$$